In [ ]:
! pip install  -r requirements.txt

In [1]:
import tensorflow as tf
import torch
from VideoSwinTransformer import *
from collections import OrderedDict
import numpy as np


In [ ]:
tf_model = tf.keras.models.load_model('/Users/mohammadshoaib/Codes/tensorflow-test/GSOC-22-Video-Swin-Transformers/swin_tiny_patch244_window877_kinetics400_1k_tf')

In [ ]:
network_function = tf.function(lambda inputs: tf_model(inputs),
                               [tf.TensorSpec(tf.TensorShape([None, 3, 8 ,224, 224]),dtype=tf.float64,
    name="x")])

# network_function = tf.function(lambda inputs: tf_model(inputs),
#                                [tf.TensorSpec(tf.keras.Input((3,8,224,224), dtype=tf.float64))])



In [ ]:
outputs = list(map(lambda tname: network_function.get_concrete_function().graph.get_tensor_by_name(tname), [
    "conv_projection"
]))

In [2]:
x_pt = torch.rand( (2,3, 8,224,224)) * 255
x_np = x_pt.numpy()
x_tf = tf.convert_to_tensor(x_np)
x_tf = tf.transpose(x_tf, perm=(0,2,3,4,1))
x_tf = tf.cast(x_tf, dtype= tf.float32)

x_tf.shape

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-08-14 18:34:05.460084: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-14 18:34:05.460561: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


TensorShape([2, 8, 224, 224, 3])

In [3]:
from VideoSwinTransformer import model_configs , SwinTransformer3D
swin = SwinTransformer3D()

input_shape = (2,3, 8,224, 224)
x = tf.keras.Input((8,224,224,3))
swin = SwinTransformer3D(shape_of_input = x.shape)
x = tf.random.normal(input_shape,   dtype="float32")
# swin = SwinTransformer3D(x)
x  = tf.keras.layers.Input(tensor=x)

output = swin(x, training= False)
print("output shape: ",output.shape)
# print(swin.get_layer("basic_layer").layers)

print(swin.input)

output shape:  (2, 768, 2, 7, 7)
KerasTensor(type_spec=TensorSpec(shape=(2, 3, 8, 224, 224), dtype=tf.float32, name=None), description="created by layer 'input_2'")


In [4]:
swin.build_graph().summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 3, 8, 224, 224)]  0         
                                                                 
 tf.compat.v1.transpose (TFO  (None, 8, 224, 224, 3)   0         
 pLambda)                                                        
                                                                 
 projection (Sequential)     (None, 2, 56, 56, 96)     18528     
                                                                 
 tf.compat.v1.transpose_1 (T  (None, 96, 2, 56, 56)    0         
 FOpLambda)                                                      
                                                                 
 dropout_49 (Dropout)        (None, 96, 2, 56, 56)     0         
                                                                 
 basic_layer_4 (BasicLayer)  (None, 192, 2, 28, 28)    320426

In [5]:
! pip install pydot graphviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 738.5 kB/s eta 0:00:00 0:00:01


In [5]:
# Just showing all possible argument for newcomer.  
tf.keras.utils.plot_model(
    swin.build_graph(),                      # here is the trick (for now)
             # saving  
    show_shapes=True, show_layer_names=True,  # show shapes and layer name
    expand_nested=False                       # will show nested block
)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [ ]:
cfg_method = model_configs.MODEL_MAP["swin_tiny_patch244_window877_kinetics400_1k"]
cfg = cfg_method()

name = cfg["name"]
link = cfg['link']
del cfg["name"]
del cfg['link']

pt_model = SwinTransformer3D_pt(**cfg)

checkpoint = torch.load(f'{name}.pth')


new_state_dict = OrderedDict()
for k, v in checkpoint['state_dict'].items():
    if 'backbone' in k:
        name = k[9:]
        new_state_dict[name] = v 

pt_model.load_state_dict(new_state_dict) 

In [ ]:
y = tf_model.predict(x_tf)


In [ ]:
z = pt_model(x_pt)

In [ ]:
y = np.round(y, 4)

In [ ]:

z.shape , y.shape


In [ ]:
y[0][0][0]

In [ ]:
z[0][0][0]

In [ ]:
np_state_dict = pt_model.state_dict()
np_state_dict = {k: np_state_dict[k].numpy() for k in np_state_dict}

In [ ]:
np.array_equal(np_state_dict['patch_embed.proj.bias'] ,tf_model.projection.layers[0].bias.numpy())

In [ ]:
i = 1
for weight in tf_model.weights:
    i +=1
    if "dense" in weight.name:
        
        isEqual = np.array_equal(weight.numpy(),np_state_dict[tf_pt[weight.name]].transpose() )
    else:
        isEqual = np.array_equal(weight.numpy(),np_state_dict[tf_pt[weight.name]] )

    if not isEqual:
        print(weight.name, weight.numpy().shape , np_state_dict[tf_pt[weight.name]].shape )
i

In [ ]:
import json

tf_pt = {}
  
# Opening JSON file
f = open('data.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
f.close()
# Iterating through the json
# list
for idx, layer in enumerate(data):
    # print(idx, layer, "------" ,  data[layer])
    tf_pt[data[layer]] = layer
  
# Closing file


# training

In [5]:
x = tf.random.normal([100,3,8, 224,224])
y = tf.random.uniform(shape=[100], minval=0, maxval=5, dtype='int64')
y

<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([1, 3, 2, 2, 4, 2, 4, 4, 4, 4, 3, 4, 4, 3, 1, 1, 0, 0, 3, 4, 4, 0,
       0, 2, 1, 3, 2, 2, 2, 1, 0, 4, 4, 3, 4, 1, 4, 0, 0, 4, 0, 2, 4, 4,
       3, 0, 1, 1, 1, 1, 2, 2, 1, 3, 0, 3, 1, 1, 1, 4, 4, 1, 2, 0, 2, 0,
       1, 3, 3, 2, 4, 0, 4, 1, 4, 0, 0, 1, 2, 0, 0, 0, 4, 3, 3, 1, 1, 0,
       4, 0, 4, 2, 3, 4, 4, 3, 0, 0, 4, 0])>

In [6]:
model = tf.keras.Sequential()
model.add(SwinTransformer3D())
model.add(tf.keras.layers.Dense(6))
model.compile(optimizer='sgd', loss='mse')
# This builds the model for the first time:
model.fit(x, y, batch_size=10, epochs=10)

Epoch 1/10
swin_transformer_block3d_24 (10, 2, 56, 56, 96)
swin_transformer_block3d_24 (10, 2, 56, 56, 96)
swin_transformer_block3d_25 (10, 2, 56, 56, 96)
swin_transformer_block3d_24 (10, 2, 56, 56, 96)
swin_transformer_block3d_24 (10, 2, 56, 56, 96)
swin_transformer_block3d_25 (10, 2, 56, 56, 96)


ValueError: in user code:

    File "/Users/mohammadshoaib/Codes/tensorflow-test/env/lib/python3.8/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/Users/mohammadshoaib/Codes/tensorflow-test/env/lib/python3.8/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/mohammadshoaib/Codes/tensorflow-test/env/lib/python3.8/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/Users/mohammadshoaib/Codes/tensorflow-test/env/lib/python3.8/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/Users/mohammadshoaib/Codes/tensorflow-test/env/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File 

In [ ]:
x  = swin(x)
#print(x.shape)

In [ ]:
# swin.summary()

In [ ]:
tf.constant(-100, dtype= 'float64')

# projection

In [ ]:
import tensorflow as tf
import torch
from keras.layers import Conv3D , LayerNormalization

In [ ]:
embed_dim = 96
patch_size = (2,7,7)

projection = tf.keras.Sequential(
[
    Conv3D(
        embed_dim ,kernel_size = patch_size , strides= patch_size , padding="valid", name= "conv_projection"
    )
],
name = "projection"
)   # data_format= "channels_first"

projection.add(LayerNormalization(epsilon=1e-5))

In [ ]:
x_pt = torch.rand( (1,3, 8,224,224)) * 255
x_np = x_pt.numpy()
x_tf = tf.convert_to_tensor(x_np)
x_tf = tf.transpose(x_tf, perm=(0,2,3,4,1))
x_tf = tf.cast(x_tf, dtype= tf.float32)

x_tf.shape